In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
import time
import copy
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_dir = '/kaggle/input/fer2013/train'
test_dir = '/kaggle/input/fer2013/test'

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("Class names:", class_names)

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0
            loader = train_loader if phase == 'train' else test_loader

            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25)


Using device: cuda:0
Class names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 203MB/s]


Epoch 0/24
----------
train Loss: 1.5651 Acc: 0.3960
val Loss: 1.3677 Acc: 0.4753

Epoch 1/24
----------
train Loss: 1.2898 Acc: 0.5037
val Loss: 1.2478 Acc: 0.5195

Epoch 2/24
----------
train Loss: 1.1882 Acc: 0.5480
val Loss: 1.1901 Acc: 0.5522

Epoch 3/24
----------
train Loss: 1.1141 Acc: 0.5771
val Loss: 1.1647 Acc: 0.5599

Epoch 4/24
----------
train Loss: 1.0562 Acc: 0.6014
val Loss: 1.1573 Acc: 0.5680

Epoch 5/24
----------
train Loss: 0.9995 Acc: 0.6223
val Loss: 1.1121 Acc: 0.5889

Epoch 6/24
----------
train Loss: 0.9505 Acc: 0.6434
val Loss: 1.1275 Acc: 0.5868

Epoch 7/24
----------
train Loss: 0.8681 Acc: 0.6751
val Loss: 1.0997 Acc: 0.5942

Epoch 8/24
----------
train Loss: 0.8401 Acc: 0.6873
val Loss: 1.1089 Acc: 0.5995

Epoch 9/24
----------
train Loss: 0.8225 Acc: 0.6939
val Loss: 1.1082 Acc: 0.6016

Epoch 10/24
----------
train Loss: 0.8140 Acc: 0.6981
val Loss: 1.0966 Acc: 0.6080

Epoch 11/24
----------
train Loss: 0.8025 Acc: 0.7006
val Loss: 1.1121 Acc: 0.6082

Ep

AttributeError: module 'torch.utils' has no attribute 'make_grid'